In [1]:
import json
import os

# Data Exploration

# G
a dictionary representing a graph as an adjacency list where： <br>
the keys are the nodes <br>
the edges are the values

In [2]:
with open(r"C:\Users\XPS\Desktop\Uni drives me crazy\Y3S1\CZ3005 Aritificial Intelligence\Project 1\G.json", encoding = "utf8") as f:
    G = json.load(f)

In [3]:
print("Number of nodes: ",len(G))
no_edges = 0
for edges in G.values():
    no_edges+=len(edges)
print("Number of edges: ",no_edges)

Number of nodes:  264346
Number of edges:  730100


# Dist
a dictionary where: <br>
keys are edges <br>
values are the distance 

In [4]:
with open(r"C:\Users\XPS\Desktop\Uni drives me crazy\Y3S1\CZ3005 Aritificial Intelligence\Project 1\Dist.json", encoding = "utf8") as f:
    Dist = json.load(f)

In [5]:
print("Number of edges: ",len(Dist))
flag = False
for d in Dist.values():
    if not isinstance(d,int) and not isinstance(d,float):
        flag = True
        break
if flag:
    print("Invalid distance detected")
else:
    print("All distances are valid")

Number of edges:  730100
All distances are valid


# Cost
a dictionary where: <br>
keys are edges <br>
values are the cost

In [6]:
with open(r"C:\Users\XPS\Desktop\Uni drives me crazy\Y3S1\CZ3005 Aritificial Intelligence\Project 1\Cost.json", encoding = "utf8") as f:
    Cost = json.load(f)

In [7]:
print("Number of edges: ",len(Cost))
flag = False
for c in Cost.values():
    if not isinstance(c,int) and not isinstance(c,float):
        flag = True
        break
if flag:
    print("Invalid Cost detected")
else:
    print("All costs are valid")

Number of edges:  730100
All costs are valid


# Coord
a dictionary where: <br>
keys are nodes <br>
values are the coordinates

In [8]:
with open(r"C:\Users\XPS\Desktop\Uni drives me crazy\Y3S1\CZ3005 Aritificial Intelligence\Project 1\Coord.json", encoding = "utf8") as f:
    Coord = json.load(f)

In [10]:
print("Number of nodes: ",len(G))
flag = False
for c in Coord.values():
    if len(c)!=2:
        flag = True
        break
if flag:
    print("Invalid coordinate detected")
else:
    print("All coordinates are valid")

Number of nodes:  264346
All coordinates are valid


# Task 1

## Priority Dictionary
The code is taken from: https://www.ics.uci.edu/~eppstein/161/python/priodict.py

In [14]:
# Priority dictionary using binary heaps
# David Eppstein, UC Irvine, 8 Mar 2002

# Implements a data structure that acts almost like a dictionary, with two modifications:
# (1) D.smallest() returns the value x minimizing D[x].
#     For this to work correctly, all values D[x] stored in the
#     dictionary must be comparable.
# (2) iterating "for x in D" finds and removes the items from D
#     in sorted order. Each item is not removed until the next item is
#     requested, so D[x] will still return a useful value until the
#     next iteration of the for-loop.
# Each operation takes logarithmic amortized time.

from __future__ import generators


class priorityDictionary(dict):
    def __init__(self):
        '''Initialize priorityDictionary by creating binary heap of
        pairs (value,key). Note that changing or removing a dict entry
        will not remove the old pair from the heap until it is found by
        smallest() or until the heap is rebuilt.'''
        self.__heap = []
        dict.__init__(self)

    def smallest(self):
        '''Find smallest item after removing deleted items from front of
        heap.'''
        if len(self) == 0:
            raise IndexError("smallest of empty priorityDictionary")
        heap = self.__heap
        while heap[0][1] not in self or self[heap[0][1]] != heap[0][0]:
            lastItem = heap.pop()
            insertionPoint = 0
            while 1:
                smallChild = 2 * insertionPoint + 1
                if smallChild + 1 < len(heap) and \
                        heap[smallChild] > heap[smallChild + 1]:
                    smallChild += 1
                if smallChild >= len(heap) or lastItem <= heap[smallChild]:
                    heap[insertionPoint] = lastItem
                    break
                heap[insertionPoint] = heap[smallChild]
                insertionPoint = smallChild
        return heap[0][1]

    def __iter__(self):
        '''Create destructive sorted iterator of priorityDictionary.'''
        def iterfn():
            while len(self) > 0:
                x = self.smallest()
                yield x
                del self[x]
        return iterfn()

    def __setitem__(self, key, val):
        '''Change value stored in dictionary and add corresponding pair
        to heap. Rebuilds the heap if the number of deleted items gets
        large, to avoid memory leakage.'''
        dict.__setitem__(self, key, val)
        heap = self.__heap
        if len(heap) > 2 * len(self):
            self.__heap = [(v, k) for k, v in self.items()]
            self.__heap.sort()
            # builtin sort probably faster than O(n)-time heapify
        else:
            newPair = (val, key)
            insertionPoint = len(heap)
            heap.append(None)
            while insertionPoint > 0 and \
                    newPair < heap[(insertionPoint - 1) // 2]:
                heap[insertionPoint] = heap[(insertionPoint - 1) // 2]
                insertionPoint = (insertionPoint - 1) // 2
            heap[insertionPoint] = newPair
            
    def setdefault(self, key, val):
        '''Reimplement setdefault to pass through our customized __setitem__.'''
        if key not in self:
            self[key] = val

## Dijkstra's algorithm
references: https://gist.github.com/hillscottc/7d2e17c59d96e26ec855 <br>
https://gist.github.com/theonewolf/6175427

In [16]:
def dijkstra_task1(graph, start, end=None):
    """Find all shortest paths from start node to a non-specific end, sg: each other node."""
    current_total_distances = {}    # dictionary of total distances so far
    candidate_nodes = {}       # dictionary of candidate nodes, where the key is neighbour node, value is current node
    estimated_total_distances = priorityDictionary()     # estimated distance of non-final nodes
    estimated_total_distances[start] = 0        # initialize the distance at start node is 0

    for current_node in estimated_total_distances:
        current_total_distances[current_node] = estimated_total_distances[current_node]
        if current_node == end:      # if we have reached the end node
            break
            
        neighbours = graph[current_node]
        # interate through every the current node's every adjacent node
        for neighbour in neighbours:
            # find the updated path_length, which is current total length + distance from current node to its neighbouring node
            path_length = current_total_distances[current_node] + Dist[f"{current_node},{neighbour}"] 
            
            # if the neighbour node has already being visited
            if neighbour in current_total_distances:
                # and the updated path_length is shorter than the previous total distance from start node to neighbour node
                # that means our current route is not the shortest
                if path_length < current_total_distances[neighbour]:
                    # raise rror
                    raise ValueError("Btter path found to an visited node!")
                    
            # if the neighbour node is not visited 
            # or if the updated path length is shorter than the estimated total distance from start node to neighbour node
            elif neighbour not in estimated_total_distances or path_length < estimated_total_distances[neighbour]:
                # update the estimated total distance from start node to neighbour as path_length
                estimated_total_distances[neighbour] = path_length
                candidate_nodes[neighbour] = current_node    
                
    return candidate_nodes, estimated_total_distances

In [17]:
def dijkstra_shortest_path(graph, start, end):
    """Find a shortest path from start node to the given end node,
    output the list of nodes visited along the shortest path,
    as well as the toal distances and total enegy costs."""
    
    candidate_nodes, estimated_total_distances = dijkstra_task1(graph, start, end)
    shortest_path = []
    while True:
        shortest_path.append(end)
        if end == start:
            break
        end = candidate_nodes[end]
    shortest_path.reverse()
    
    # print out the shortest path
    print("Shortest path:",end = '')
    for i in range(0, len(shortest_path)-1):
        print(shortest_path[i], "->", end = '')
    print(shortest_path[-1], ".") 
    
    # obtain the value in the estimtaed_total_distances dictionary, the ned node is the key
    print("Shortest distance:", estimated_total_distances[shortest_path[-1]],".")
    
    # calculate total energy
    total_energy = 0
    for i in range(0, len(shortest_path)-1):
        total_energy += Cost[f"{shortest_path[i]},{shortest_path[i+1]}"] 
    print("Total energy cost:", total_energy, ".")
    # Task 1 does not satisfy the energy constraint

In [18]:
start = "1"
end = "50"
dijkstra_shortest_path(G, start, end)

Shortest path:1 ->1363 ->1358 ->1357 ->1356 ->1276 ->1273 ->1277 ->1269 ->1267 ->1268 ->1284 ->1283 ->1282 ->1255 ->1253 ->1260 ->1259 ->1249 ->1246 ->963 ->964 ->962 ->1002 ->952 ->1000 ->998 ->994 ->995 ->996 ->987 ->988 ->979 ->980 ->969 ->977 ->989 ->990 ->991 ->2369 ->2366 ->2340 ->2338 ->2339 ->2333 ->2334 ->2329 ->2029 ->2027 ->2019 ->2022 ->2000 ->1996 ->1997 ->1993 ->1992 ->1989 ->1984 ->2001 ->1900 ->1875 ->1874 ->1965 ->1963 ->1964 ->1923 ->1944 ->1945 ->1938 ->1937 ->1939 ->1935 ->1931 ->1934 ->1673 ->1675 ->1674 ->1837 ->1671 ->1828 ->1825 ->1817 ->1815 ->1634 ->1814 ->1813 ->1632 ->1631 ->1742 ->1741 ->1740 ->1739 ->1591 ->1689 ->1585 ->1584 ->1688 ->1579 ->1679 ->1677 ->104 ->5680 ->5418 ->5431 ->5425 ->5424 ->5422 ->5413 ->5412 ->5411 ->66 ->5392 ->5391 ->5388 ->5291 ->5278 ->5289 ->5290 ->5283 ->5284 ->5280 ->50 .
Shortest distance: 148648.63722140007 .
Total energy cost: 294853 .
